<center>
<h1> TP1 et 2- Recherche Operationnelle </h1>
<h1> Année 2020-2021 - 2e année département Sciences du Numérique </h1>
<h1> Noms:  Ghanim - Louahabi</h1>
<h1> Prénoms:  Ahmed - Chaimaa</h1>    
</center>

In [15]:
#importer les packages utiles, le manager de package Pkg etant un package
#import Pkg; Pkg.add("Cbc")
#Pkg.add("JuMP")

# Applications en optimisation pour l’e-commerce
Etude## Cas particulier 1

In [ ]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
nb_mag = 3 # nombre de magasins disponibles
nb_fl = 2 # nombre de fluides
nb_dem = 2 #nombre de demandes/clients
C = [1 1;2 3; 3 2] # Coûts  unitaires  de  disponibilité par magasin (Cij coût de Produit_j au magasin i)
S = [2.5 1; 1 2; 2 1] # Stocks de produits par magasin
D = [2 0; 1 3] # Produits demandés par commande

# set optimizer
model = Model(Cbc.Optimizer)

# Q[i,j,k] la quantité de fluide j prise du magasin i pour le client k
@variable(model, Q[1:nb_mag, 1:nb_fl , 1:nb_dem] >= 0)

# define objective function
@objective(model, Min, sum(sum(sum(C[i,j]*Q[i,j,k] for i in 1:nb_mag) for j in 1:nb_fl) for k in 1:nb_dem))

# Chaque demande doit être satisfaite dans sa totalité.
for k in 1 : nb_dem
    for j in 1:nb_fl
        @constraint(model, sum(Q[i,j,k] for i in 1:nb_mag) == D[k,j])
    end
end
#La quantité d'un fluide prise d'un magasin ne peut pas dépasser la quantité disponible au stock
for i in 1:nb_mag 
    for j in 1:nb_fl
        @constraint(model, sum(Q[i,j,k] for k in 1:nb_dem)  <= S[i,j])
    end
end
# run optimization
optimize!(model)

In [1]:
# print solution
println("Solution obtenue:")
println("\t Benefice = $(objective_value(model))")
for k in 1:nb_dem
    println("La commande du client $k :")
    for j in 1:nb_fl
        for i in 1:nb_mag
            println("\t Quantité de fluide$j du magasin $i pour le client $k = $(value(Q[i, j, k]))")
        end
        println("---------------")
    end
end

Solution obtenue:


UndefVarError: UndefVarError: objective_value not defined

### Résultats du cas particulier 1

Pour le jeu de données du sujet, le bénifice est 9,5. Le résultat parraît raisonnalbe vue que -par exemple- le stock du premier magasin a bien été épuisé car ses coûts sont les plus faibles.

## Cas particulier 2

On garde les mêmes données et variables que le premier cas particulier, mais vue que maintenant on ne parle plus de fluides mais de produits, alors le problème devient PLNE. La variable $Q_{ijk}$ est devenue de type entier.

In [32]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
nb_mag = 3 # nombre de magasins disponibles
nb_prod = 2 # nombre de produits
nb_dem = 2 #nombre de demandes/clients
C = [1 1;2 3; 3 2] # Coûts  unitaires  de  disponibilité par magasin (Cij cout de Produit_j au magasin i)
S = [2.5 1; 1 2; 2 1] # Stocks de produits par magasin
D = [2 0; 1 3] # Produits demandés par commande

# set optimizer
model = Model(Cbc.Optimizer)

# Q[i,j,k] la quantité du produit j prise du magasin i pour le client k
@variable(model, Q[1:nb_mag, 1:nb_prod , 1:nb_dem] >= 0, Int)

# define objective function
@objective(model, Min, sum(sum(sum(C[i,j]*Q[i,j,k] for i in 1:nb_mag) for j in 1:nb_prod) for k in 1:nb_dem))

# Chaque demande doit être satisfaite dans sa totalité.
for k in 1 : nb_dem
    for j in 1:nb_prod
        @constraint(model, sum(Q[i,j,k] for i in 1:nb_mag) == D[k,j])
    end
end
#La quantité d'un fluide prise d'un magasin ne peut pas dépasser la quantité disponible au stock
for i in 1:nb_mag 
    for j in 1:nb_prod
        @constraint(model, sum(Q[i,j,k] for k in 1:nb_dem)  <= S[i,j])
    end
end
# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 9.5 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 6 rows, 9 columns (9 integer (6 of which binary)) and 17 elements
Cbc0012I Integer solution of 10 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 10, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 10 to 10
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created

In [16]:
# print solution
println("Solution obtenue:")
println("\t Benefice = $(objective_value(model))")
for k in 1:nb_dem
    println("La commande du client $k :")
    for j in 1:nb_prod
        for i in 1:nb_mag
            println("\t Quantité du produit$j du magasin $i pour le client $k = $(value(Q[i, j, k]))")
        end
        println("---------------")
    end
end

Solution obtenue:
	 Benefice = 10.0
La commande du client 1 :
	 Quantité du produit1 du magasin 1 pour le client 1 = 1.0
	 Quantité du produit1 du magasin 2 pour le client 1 = 1.0
	 Quantité du produit1 du magasin 3 pour le client 1 = 0.0
---------------
	 Quantité du produit2 du magasin 1 pour le client 1 = 0.0
	 Quantité du produit2 du magasin 2 pour le client 1 = 0.0
	 Quantité du produit2 du magasin 3 pour le client 1 = 0.0
---------------
La commande du client 2 :
	 Quantité du produit1 du magasin 1 pour le client 2 = 1.0
	 Quantité du produit1 du magasin 2 pour le client 2 = 0.0
	 Quantité du produit1 du magasin 3 pour le client 2 = 0.0
---------------
	 Quantité du produit2 du magasin 1 pour le client 2 = 1.0
	 Quantité du produit2 du magasin 2 pour le client 2 = 1.0
	 Quantité du produit2 du magasin 3 pour le client 2 = 1.0
---------------


### Résultats du cas particulier 2

Le bénéfice devient 10 dans le PLNE, ce qui est raisonnable car le client 1 est obligé de prendre un produit de chez le deuxième magasin qui a un coût plus élevé que le 1er magasin.

## Comparaison Temps de Calcul entre PL et PLNE

Si on teste les deux cas particuliers 1 et 2 avec 10 magasins, 10 produits et 5 demandes, on voit que le temps de calcul pour PL (cas p. 1) est toujours 0.00 alors que le deuxième cas particulier (PLNE) prend 0.01. Donc plus la taille des problèmes est grande, plus les problèmes PLNE sont lents.

In [5]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
nb_mag = 10 # nombre de magasins disponibles
nb_fl = 10 # nombre de fluides
nb_dem = 5 #nombre de demandes/clients
C = [1 0 0 0 1 1 1 2 2 1 ;
     5 4 2 0 0 0 0 1 2 3 ;
     3 2 0 0 0 0 0 0 0 0 ;
     2 0 1 1 2 4 8 7 6 4 ;
     2 3 4 6 7 4 4 2 2 5 ;
     4 5 4 0 0 1 0 2 2 2 ;
     1 0 0 0 1 1 1 2 2 1 ;
     5 4 2 0 0 0 0 1 2 3 ;
     3 2 0 0 0 0 0 0 0 0 ;
     2 0 1 1 2 4 8 7 6 4 ] # Coûts  unitaires  de  disponibilité par magasin (Cij coût de Produit_j au magasin i)
S = [3 1 0 1 1 0 1 2 1 0 ; 
    0 0 0 0 1 0 0 0 0 0 ; 
    1 2 0 0 1 0 1 1 1 1 ; 
    1 1 1 0 2 0 2 0 0 0 ; 
    1 0 0 3 0 1 1 0 2 0 ; 
    1 1 2 0 2 0 0 0 1 0 ; 
    1 1 3 3 2 0 1 0 0 0 ; 
    0 0 2 1 0 0 0 2 0 1 ; 
    1 2 1 1 0 2 0 0 3 0 ; 
    0 1 0 0 1 0 1 0 1 0 ] # Stocks de produits par magasin
D = [0 3 0 3 3 0 2 0 2 0 ; 
    0 0 0 0 1 1 0 3 1 0 ; 
    4 0 0 0 4 0 0 0 0 0 ; 
    0 4 4 0 0 0 0 0 4 0 ; 
    3 0 3 4 0 0 3 0 0 0 ] # Produits demandés par commande

# set optimizer
model = Model(Cbc.Optimizer)

# Q[i,j,k] la quantité de fluide j prise du magasin i pour le client k
@variable(model, Q[1:nb_mag, 1:nb_fl , 1:nb_dem] >= 0, Int)

# define objective function
@objective(model, Min, sum(sum(sum(C[i,j]*Q[i,j,k] for i in 1:nb_mag) for j in 1:nb_fl) for k in 1:nb_dem))

# Chaque demande doit être satisfaite dans sa totalité.
for k in 1 : nb_dem
    for j in 1:nb_fl
        @constraint(model, sum(Q[i,j,k] for i in 1:nb_mag) == D[k,j])
    end
end
#La quantité d'un fluide prise d'un magasin ne peut pas dépasser la quantité disponible au stock
for i in 1:nb_mag 
    for j in 1:nb_fl
        @constraint(model, sum(Q[i,j,k] for k in 1:nb_dem)  <= S[i,j])
    end
end
# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 55 - 0.00 seconds
Cgl0004I processed model has 60 rows, 101 columns (101 integer (71 of which binary)) and 200 elements
Cbc0012I Integer solution of 55 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 55, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 55 to 55
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active af

## Cas particulier 3

### Modélisation :
On garde toutes les données et variables d'avant et on introduit une nouvelle variable binaire $X_{ij} $ qui vaut 1 si le client i prend au moins un produit du magasin j, et 0 sinon. On introduit aussi la matrice des coûts d'expédition et le vecteur M qui nous servira pour appliquer la méthode du Big-M.

In [36]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
nb_mag = 3 # nombre de magasins disponibles
nb_dem = 2 #nombre de demandes/clients
nb_pro = 2 #nombre de produits
Cdisp = [1 1;2 3; 3 2] # Coûts  unitaires  de  disponibilité par magasin (Cij cout de Produit j au magasin i)
S = [2.5 1; 1 2; 2 1] # Stocks de produits par magasin
D = [2 0; 1 3] # Produits demandés par commande
Cexp = [1 0 10; 0 2 10] #Côuts d’expédition d’un colis entre chaque paire (point de demande, magasin)
M = [sum(D[1,j] for j in 1:nb_pro); sum(D[2,j] for j in 1:nb_pro)] # pour la méthode du big-M
# set optimizer
model = Model(Cbc.Optimizer)

# Q[i,j,k] la quantité du produit j prise du magasin i pour le client k
@variable(model, Q[1:nb_mag, 1:nb_pro , 1:nb_dem] >= 0, Int)
#X_ij vaut 1 si le client i prend au moins 1 produit du magasin j, 0 sinon.
@variable(model, x[1:nb_dem, 1:nb_mag], Bin)

# define objective function
@objective(model, Min, sum(sum(sum(Cdisp[i,j]*Q[i,j,k] for i in 1:nb_mag) for j in 1:nb_prod) for k in 1:nb_dem) + sum(Cexp .* x))

# define constraints
for k in 1 : nb_dem
    for j in 1:nb_prod
        @constraint(model, sum(Q[i,j,k] for i in 1:nb_mag) == D[k,j])
    end
end

for i in 1:nb_mag 
    for j in 1:nb_prod
        @constraint(model, sum(Q[i,j,k] for k in 1:nb_dem)  <= S[i,j])
    end
end                                              

for k in 1:nb_dem 
    for i in 1:nb_mag
        #pour imposer 1 sur x si le client k prend au moins un produit du magasin i
        @constraint(model, sum(Q[i,j,k] for j in 1:nb_prod) <= M[k]*x[k,i])
        #pour imposer zero sur x si le client k ne prend aucun produit du magasin i
        @constraint(model, x[k,i] <= sum(Q[i,j,k] for j in 1:nb_prod) )
    end
end
                                                                                    
# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 12.25 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 3 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 14 rows, 13 columns (13 integer (10 of which binary)) and 38 elements
Cbc0012I Integer solution of 14 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 14, took 2 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 4 variables fixed on reduced cost
Cuts at root node changed objective from 13.5 to 14
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and

In [37]:
println("Solution obtenue:")
println("\t Benefice = $(objective_value(model))")
for k in 1:nb_dem
    println("La commande du client $k :")
    for j in 1:nb_prod
        for i in 1:nb_mag
            println("\t Quantité du produit$j du magasin $i pour le client $k = $(value(Q[i, j, k]))")
        end
        println("---------------")
    end
end
println("Vérification de binaires Xij ")
for i in 1:nb_dem
    for j in 1:nb_mag
        println("\t Le client $i a pris au moins un produit du magasin $j = $(value(x[i, j]))")
    end
    println("---------------")
end


Solution obtenue:
	 Benefice = 14.0
La commande du client 1 :
	 Quantité du produit1 du magasin 1 pour le client 1 = 1.0
	 Quantité du produit1 du magasin 2 pour le client 1 = 1.0
	 Quantité du produit1 du magasin 3 pour le client 1 = 0.0
---------------
	 Quantité du produit2 du magasin 1 pour le client 1 = 0.0
	 Quantité du produit2 du magasin 2 pour le client 1 = 0.0
	 Quantité du produit2 du magasin 3 pour le client 1 = 0.0
---------------
La commande du client 2 :
	 Quantité du produit1 du magasin 1 pour le client 2 = 1.0
	 Quantité du produit1 du magasin 2 pour le client 2 = 0.0
	 Quantité du produit1 du magasin 3 pour le client 2 = 0.0
---------------
	 Quantité du produit2 du magasin 1 pour le client 2 = 1.0
	 Quantité du produit2 du magasin 2 pour le client 2 = 2.0
	 Quantité du produit2 du magasin 3 pour le client 2 = 0.0
---------------
Vérification de binaires Xij 
	 Le client 1 a pris au moins un produit du magasin 1 = 1.0
	 Le client 1 a pris au moins un produit du magasi

### Résultats Cas Particulier 3 

Le bénéfice après l'introduction des coûts de livraison est passé de 10 à 14, ce qui correspont simplement à l'ajout des frais de livraison (vers les deux premiers magasins pour client1, et vers les trois magasins pour le client 2).On remarque que on epuise toujours le stock du magasin 1 car même avec un coût d'expédition nul, les magasins 2 et 3 restent plus coûteux en terme de coûts unitaires.
Si on remplace les coûts d'expédition du magasin (vers les deux clients) par un grand nombre (10 par exemple) , on remarque que les deux clients ne prennent aucun produit du magasin 3 . Ce qui est raisonnable.

# Le Problème Générale

## Modélisation

On a introduit la variable $M_{ijk}$ (pour Map), qui vaut 1 si l'arc orienté (i,j) fait partie de la tournée du livreur du magasin k, et 0 sinon. On a gardé la variable $X_{ij}$ pour décider si le livreur j doit passer par le client i ou pas. L'objectif est de minimiser la distance totale parcourue par tous les livreurs.

In [2]:
using Cbc
using JuMP
include("2_4.jl")
inputfilepath = "data_2-4"
inputfilename = "Data_test_5_1_2.txt" #"Data_test_5_2_3.txt" 
nb_dem, nb_prod, nb_mag, nb_noeuds, S, Q, R = read_data_24(inputfilepath, inputfilename)
# set optimizer
model = Model(Cbc.Optimizer)
# define variables
# quantités des produits Livrées par chaque magasin i du produit j au client k
@variable(model, L[1:nb_mag, 1:nb_prod , 1:nb_dem] >= 0, Int) 
@variable(model, X[1:nb_dem, 1:nb_mag], Bin)
#map
@variable(model, M[1:nb_noeuds, 1:nb_noeuds , 1:nb_mag] >= 0, Bin)
# define objective function
@objective(model, Min, sum(sum(sum(R[i][j]*M[i,j,k] for i in 1:nb_noeuds) for j in 1:nb_noeuds) for k in 1:nb_mag))

                            
# define constraints
for k in 1 : nb_dem
    for j in 1:nb_prod
        @constraint(model, sum(L[i,j,k] for i in 1:nb_mag) == Q[k][j])
    end
end
#contrainte sur le stock
for i in 1:nb_mag
    for j in 1:nb_prod
        @constraint(model, sum(L[i,j,k] for k in 1:nb_dem)  <= S[i][j])
    end
end
#contrainte sur le biniare X
for k in 1:nb_dem 
    for i in 1:nb_mag
        @constraint(model, sum(L[i,j,k] for j in 1:nb_prod) <= sum(Q[k][j] for j in 1:nb_prod)*X[k,i])
        @constraint(model, X[k,i] <= sum(L[i,j,k] for j in 1:nb_prod))
    end
end
#contrainte sur les maps
for k in 1:nb_mag
    #le magasin fait necessairement partie de la tournée de son livreur
    @constraint(model,sum(M[i,k,k] for i in 1:nb_noeuds) == 1) 
    for j in (1+nb_mag):nb_noeuds
        @constraint(model,sum(M[i,j,k] for i in 1:nb_noeuds) == X[j-nb_mag, k])
    end
end
#Pour ne pas sléctionner l'arc entre un noeud et soi même.
for k in 1:nb_mag
    @constraint(model, sum(M[i,i,k] for i in 1:nb_noeuds) == 0)
end                                                                        
#le livreur ne passe pas par les autres magasins
for k in 1:nb_mag
    for i in 1:k-1
        @constraint(model,sum(M[i,j,k] for j in 1:nb_noeuds) == 0)
        @constraint(model,sum(M[j,i,k] for j in 1:nb_noeuds) == 0)
    end
    for i in k+1:nb_mag
        @constraint(model,sum(M[i,j,k] for j in 1:nb_noeuds) == 0)
        @constraint(model,sum(M[j,i,k] for j in 1:nb_noeuds) == 0)
    end
end
for k in 1:nb_mag
    #le magasin fait necessairement partie de la tournée de son livreur
    @constraint(model,sum(M[k,j,k] for j in 1:nb_noeuds) == 1) 
    for i in (1+nb_mag):nb_noeuds
        @constraint(model,sum(M[i,j,k] for j in 1:nb_noeuds) == X[i-nb_mag, k])
    end
end

#Contrainte: Pas de sous tours
for k in 1:nb_mag
    for i in nb_mag +1:nb_noeuds-1
        for j in i+1:nb_noeuds
            @constraint(model,M[i,j,k]+M[j,i,k]<=1)
        end
    end
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 6 - 0.00 seconds
Cgl0002I 6 variables fixed
Cgl0004I processed model has 22 rows, 30 columns (30 integer (30 of which binary)) and 80 elements
Cbc0012I Integer solution of 6 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 6, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 6 to 6
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of whi

In [3]:
println("Solution obtenue:")
println("\t Benefice = $(objective_value(model))")
for k in 1:nb_dem
    #println("La commande du client $k :")
    for j in 1:nb_prod
        for i in 1:nb_mag
            #println("\t Quantité du produit$j du magasin $i pour le client $k = $(value(L[i, j, k]))")
        end
        #println("---------------")
    end
end
println("Tour des livreurs ---------------")
for k in 1:nb_mag
    for j in 1:nb_noeuds
        for i in 1:nb_noeuds
            println("\t M $(i) _ $(j) _$k  = $(value(M[i, j, k]))")
        end
    end
end
println("Les clients de chaque livreur ---------------")
        for i in 1:nb_dem
            for j in 1:nb_mag
                println("\t Le client $i a pris au moins un produit du magasin $j = $(value(x[i, j]))")
            end
        end

Solution obtenue:
	 Benefice = 6.0
Tour des livreurs ---------------
	 M 1 _ 1 _1  = 0.0
	 M 2 _ 1 _1  = 0.0
	 M 3 _ 1 _1  = 1.0
	 M 4 _ 1 _1  = 0.0
	 M 5 _ 1 _1  = 0.0
	 M 6 _ 1 _1  = 0.0
	 M 1 _ 2 _1  = 1.0
	 M 2 _ 2 _1  = 0.0
	 M 3 _ 2 _1  = 0.0
	 M 4 _ 2 _1  = 0.0
	 M 5 _ 2 _1  = 0.0
	 M 6 _ 2 _1  = 0.0
	 M 1 _ 3 _1  = 0.0
	 M 2 _ 3 _1  = 1.0
	 M 3 _ 3 _1  = 0.0
	 M 4 _ 3 _1  = 0.0
	 M 5 _ 3 _1  = 0.0
	 M 6 _ 3 _1  = 0.0
	 M 1 _ 4 _1  = 0.0
	 M 2 _ 4 _1  = 0.0
	 M 3 _ 4 _1  = 0.0
	 M 4 _ 4 _1  = 0.0
	 M 5 _ 4 _1  = 0.0
	 M 6 _ 4 _1  = 0.9999999999999999
	 M 1 _ 5 _1  = 0.0
	 M 2 _ 5 _1  = 0.0
	 M 3 _ 5 _1  = 0.0
	 M 4 _ 5 _1  = 0.9999999999999999
	 M 5 _ 5 _1  = 0.0
	 M 6 _ 5 _1  = 0.0
	 M 1 _ 6 _1  = 0.0
	 M 2 _ 6 _1  = 0.0
	 M 3 _ 6 _1  = 0.0
	 M 4 _ 6 _1  = 0.0
	 M 5 _ 6 _1  = 0.9999999999999999
	 M 6 _ 6 _1  = 0.0
Les clients de chaque livreur ---------------


UndefVarError: UndefVarError: x not defined

## Difficultés Rencontrées Au Problème Générale

Pour l'élimination des sous tours, la contrainte qu'on a voulu exprimée en Julia est : quel que soit i et j dans S un sous ensemble de D (l'ensemble des noeuds de clients) $\sum_{i,j}M_{ijk}$ <= |S| -1 , cette contrainte est  utillisée dans la résolution  de Traveling Salesman Problem, et suppose que l'inégalité triangulaire est vérifiée.
On a pas arrivé à la traduire en Julia, vue qu'il faut tester toutes la partitions de D.
On l'as appliqué pour éliminer les sous tours entre deux clients, ce qui fait que le cas 5_1_2 marche bien, mais dès qu'on passe au cas 5_2_3, on voit apparaître des sous tours de 3 clients.